In [2]:
import requests,json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
import time

2021-09-26 15:40:46.040140: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-09-26 15:40:46.040165: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def hist_price_dl(coin_id='Qwsogvtv82FCd',timeframe = "5y",currency = "USD"):
    headers_dict = {"x-access-token": "your-api-token"}
    data = requests.get("https://api.coinranking.com/v2/coin/{}/history?timePeriod=5y".format(coin_id), headers=headers_dict)
    data = json.loads(data.text)['data']['history']
    df = pd.DataFrame(data[0:len(data)-1])
    df['price'] = pd.to_numeric(df['price'])
    df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s').dt.date

    return df.groupby('timestamp').mean()['price']

In [4]:
def price_matrix_creator(data, seq_len=30):
    '''
    It converts the series into a nested list where every item of the list contains historic prices of 30 days
    '''
    price_matrix = []
    for index in range(len(data)-seq_len+1):
        price_matrix.append(data[index:index+seq_len])
    return price_matrix


def normalize_windows(window_data):
    '''
    It normalizes each value to reflect the percentage changes from starting point
    '''
    normalised_data = []
    for window in window_data:
        normalised_window = [((float(p) / float(window[0])) - 1) for p in window]
        normalised_data.append(normalised_window)
    return normalised_data

def train_test_split_(price_matrix, train_size=0.9, shuffle=False, return_row=True):
    '''
    It makes a custom train test split where the last part is kept as the training set.
    '''
    price_matrix = np.array(price_matrix)
    # select 90% of our data for training the network
    row = int(round(train_size * len(price_matrix)))
    # slice training data
    train = price_matrix[:row, :]
    if shuffle==True:
        np.random.shuffle(train)
    # the last index in nested list is output of that period y_train
    # len - 1 is x_train and training data and y_train is output of that data
    X_train, y_train = train[:row,:-1], train[:row,-1]
    # the 10% of data is our test data
    X_test, y_test = price_matrix[row:,:-1], price_matrix[row:,-1]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    if return_row:
        return row, X_train, y_train, X_test, y_test
    else:
        return X_train, y_train, X_test, y_test

In [5]:
data = hist_price_dl()
data_1 = price_matrix_creator(data)
data_2 = normalize_windows(data_1)
row, X_train, y_train, X_Test, y_test = train_test_split_(data_2)

In [6]:
# LSTM Model parameters, I chose
batch_size = 2            # Batch size (you may try different values)
epochs = 15               # Epoch (you may try different values)
seq_len = 30              # 30 sequence data (Representing the last 30 days)
loss='mean_squared_error' # Since the metric is MSE/RMSE
optimizer = 'rmsprop'     # Recommended optimizer for RNN
activation = 'linear'     # Linear activation
input_shape=(None,1)      # Input dimension
output_dim = 30           # Output dimension

In [7]:
model = Sequential()
model.add(LSTM(units=output_dim, return_sequences=True, input_shape=input_shape))
model.add(Dense(units=32,activation=activation))
model.add(LSTM(units=output_dim, return_sequences=False))
model.add(Dense(units=1,activation=activation))
model.compile(optimizer=optimizer,loss=loss)

2021-09-26 15:40:54.445222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-26 15:40:54.445278: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-26 15:40:54.445303: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (imac): /proc/driver/nvidia/version does not exist


In [8]:
start_time = time.time()
model.fit(x=X_train,
          y=y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.05)
end_time = time.time()
processing_time = end_time - start_time
processing_time

2021-09-26 15:40:55.137087: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15
767/767 [==============================] - 14s 14ms/step - loss: 0.0080 - val_loss: 0.0203
Epoch 2/15
767/767 [==============================] - 10s 13ms/step - loss: 0.0037 - val_loss: 0.0053
Epoch 3/15
767/767 [==============================] - 10s 13ms/step - loss: 0.0027 - val_loss: 0.0055
Epoch 4/15
767/767 [==============================] - 10s 13ms/step - loss: 0.0024 - val_loss: 0.0041
Epoch 5/15
767/767 [==============================] - 10s 13ms/step - loss: 0.0024 - val_loss: 0.0046
Epoch 6/15
767/767 [==============================] - 11s 14ms/step - loss: 0.0023 - val_loss: 0.0043
Epoch 7/15
767/767 [==============================] - 10s 13ms/step - loss: 0.0025 - val_loss: 0.0076
Epoch 8/15
767/767 [==============================] - 10s 14ms/step - loss: 0.0024 - val_loss: 0.0050
Epoch 9/15
767/767 [==============================] - 10s 13ms/step - loss: 0.0022 - val_loss: 0.0046
Epoch 10/15
767/767 [==============================] - 10s 13ms/step - loss: 0.002

156.46561884880066

In [9]:
model.save('coin_predictor.h5')

In [10]:
from keras.models import load_model
model = load_model('coin_predictor.h5')
preds = model.predict(X_Test, batch_size=2)

In [11]:
def deserializer(preds, ser, train_size=0.9, train_phase=False):
    '''
    Arguments:
    preds : Predictions to be converted back to their original values
    data : It takes the data into account because the normalization was made based on the full historic data
    train_size : Only applicable when used in train_phase
    train_phase : When a train-test split is made, this should be set to True so that a cut point (row) is calculated based on the train_size argument, otherwise cut point is set to 0
    
    Returns:
    A list of deserialized prediction values, original true values, and date values for plotting
    '''
    price_matrix = np.array(price_matrix_creator(ser))
    if train_phase:
        row = int(round(train_size * len(price_matrix)))
    else:
        row=0
    date = ser.index[row+29:]
    date = np.reshape(date, (date.shape[0]))
    X_test = price_matrix[row:,:-1]
    y_test = price_matrix[row:,-1]
    preds_original = []
    preds = np.reshape(preds, (preds.shape[0]))
    for index in range(0, len(preds)):
        pred = (preds[index]+1)* X_test[index][0]
        preds_original.append(pred)
    preds_original = np.array(preds_original)
    if train_phase:
        return [date, y_test, preds_original]
    else:
        import datetime
        return [date+datetime.timedelta(days=1),y_test]

In [12]:
plotlist = deserializer(preds, data, train_size=0.9, train_phase=True)
print(plotlist[1][-1], plotlist[0][-1], plotlist[2][-1])

42702.818290178046 2021-09-26 42344.73435327261


In [13]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks as cf
init_notebook_mode(connected=True)

prices = pd.DataFrame({'Predictions':plotlist[1], 'Real Prices':plotlist[2]},index=plotlist[0])
iplot(prices.iplot(asFigure=True,
                   kind='scatter',
                   xTitle='Date',
                   yTitle='BTC Price',
                   title='BTC Price Predictions'))